In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import random
import sys
import glob
sys.path.append("./")

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
from signet.configs.Conv1D_LSTM_CTC_Loss import Conv1D_LSTM_CTC_Loss
CFG = Conv1D_LSTM_CTC_Loss()

In [4]:
# Seed all random number generators
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
def get_strategy():
    if CFG.device == "GPU"  or CFG.device=="CPU":
        ngpu = len(tf.config.experimental.list_physical_devices('GPU'))
        if ngpu>1:
            print("Using multi GPU")
            strategy = tf.distribute.MirroredStrategy()
        elif ngpu==1:
            print("Using single GPU")
            strategy = tf.distribute.get_strategy()
        else:
            print("Using CPU")
            strategy = tf.distribute.get_strategy()
            CFG.device = "CPU"

    AUTO     = tf.data.experimental.AUTOTUNE
    REPLICAS = strategy.num_replicas_in_sync
    print(f'REPLICAS: {REPLICAS}')
    
    return strategy, REPLICAS

STRATEGY, N_REPLICAS = get_strategy()

Using single GPU
REPLICAS: 1


In [5]:
ALL_FILENAMES = glob.glob('../dataset/tdf_data/*.tfrecords')
print(len(ALL_FILENAMES))

120165


In [6]:
ROOT_DIR = "../dataset"
supp_df = pd.read_csv("../dataset/supplemental_metadata.csv")
train_df = pd.read_csv("../dataset/train.csv")

In [7]:
from signet.dataset.utils import get_tfrec_dataset

In [8]:
ds = get_tfrec_dataset(ALL_FILENAMES,CFG)

<unknown>


ValueError: in user code:

    /app/ThesisProject/SigNet/signet/dataset/utils.py:41 None  *
        ds = ds.map(lambda x: preprocess(x, CFG), tf.data.experimental.AUTOTUNE)
    /app/ThesisProject/SigNet/signet/dataset/utils.py:15 preprocess  *
        coord = filter_nans_tf(coord,CFG)
    /app/ThesisProject/SigNet/signet/dataset/utils.py:9 filter_nans_tf  *
        x = tf.boolean_mask(x, mask, axis=0)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/util/dispatch.py:201 wrapper  **
        return target(*args, **kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/array_ops.py:1803 boolean_mask_v2
        return boolean_mask(tensor, mask, name, axis)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/util/dispatch.py:201 wrapper
        return target(*args, **kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/array_ops.py:1724 boolean_mask
        "Number of mask dimensions must be specified, even if some dimensions"

    